In [ ]:
!pip install yt-dlp
!pip install moviepy
!pip install torch torchvision torchaudio
!pip install git+https://github.com/openai/whisper.git



  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-2pqy9aqh
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-2pqy9aqh
  Resolved https://github.com/openai/whisper.git to commit 5979f03701209bb035a0a466f14131aeb1116cbb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803560 sha256=4cd37561d984375c904c163775e99cf65f09e71e6c0a50d5bdcb153e1f2e4b70
  Stored in directory: /tmp/pip-ephem-wheel-cache-qgilbi5b/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [ ]:
from yt_dlp import YoutubeDL
from moviepy.editor import VideoFileClip
import os
from glob import glob
import whisper
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from transformers import BertTokenizer, EncoderDecoderModel
from nltk.corpus import stopwords
import re

### LIST of video that was in the paper

In [ ]:
video_url = ['https://youtu.be/fC7V8QsPBec?si=4ooByD-C_Kdlrwh2','https://youtu.be/a2B69vCtjOU?si=e4V3gGPYSnuNT7p7',"https://youtu.be/otN3V0beVRo?si=-Zq2eyfFalDmES31","https://youtu.be/Gm4W6BNqDB4?si=06oKDIFz9zbjkfZG"]


### Download it from youtube

In [ ]:
def download_youtube_video_yt_dlp(url, output_path='/content/videos/'):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': f'{output_path}/%(title)s.%(ext)s'
    }

    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

In [ ]:
for i in video_url:
  download_youtube_video_yt_dlp(i)

[youtube] Extracting URL: https://youtu.be/fC7V8QsPBec?si=4ooByD-C_Kdlrwh2
[youtube] fC7V8QsPBec: Downloading webpage
[youtube] fC7V8QsPBec: Downloading ios player API JSON
[youtube] fC7V8QsPBec: Downloading mweb player API JSON
[youtube] fC7V8QsPBec: Downloading player 4e23410d
[youtube] fC7V8QsPBec: Downloading m3u8 information
[info] fC7V8QsPBec: Downloading 1 format(s): 609+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 181
[download] Destination: /content/videos//Week 1 - Lecture 1 - Introduction to Machine Learning.f609.mp4
[download] 100% of  110.34MiB in 00:00:21 at 5.11MiB/s                   
[download] Destination: /content/videos//Week 1 - Lecture 1 - Introduction to Machine Learning.f140.m4a
[download] 100% of   14.32MiB in 00:00:01 at 7.29MiB/s   
[Merger] Merging formats into "/content/videos//Week 1 - Lecture 1 - Introduction to Machine Learning.mp4"
Deleting original file /content/videos//Week 1 - Lecture 1 - Introduction to Machine Learning.f14

### Extract the voice from the video

In [ ]:
video_path = glob("/content/videos/*")

In [ ]:
video_path

['/content/videos/W1 L1 - Introduction to OS.mp4',
 '/content/videos/Week 1 - Lecture 1 - Introduction to Machine Learning.mp4',
 '/content/videos/Recoverable schdeules and transaction isolation levels.mp4',
 '/content/videos/Part1 - Definition of Information Security.mp4']

In [ ]:
output_Audio='/content/Audio/'
if not os.path.exists(output_Audio):
        os.makedirs(output_Audio)

In [ ]:

for path in video_path:
    try:
        audio_output_path = path.replace("videos", "Audio").replace(".mp4", ".mp3")

        output_dir = os.path.dirname(audio_output_path)
        os.makedirs(output_dir, exist_ok=True)

        video_clip = VideoFileClip(path)
        audio_clip = video_clip.audio

        audio_clip.write_audiofile(audio_output_path)

        audio_clip.close()
        video_clip.close()

        print(f"Audio extracted successfully for {path}.")

    except Exception as e:
        print(f"Failed to extract audio for {path}. Error: {e}")


MoviePy - Writing audio in /content/Audio/W1 L1 - Introduction to OS.mp3


MoviePy - Done.
Audio extracted successfully for /content/videos/W1 L1 - Introduction to OS.mp4.
MoviePy - Writing audio in /content/Audio/Week 1 - Lecture 1 - Introduction to Machine Learning.mp3


MoviePy - Done.
Audio extracted successfully for /content/videos/Week 1 - Lecture 1 - Introduction to Machine Learning.mp4.
MoviePy - Writing audio in /content/Audio/Recoverable schdeules and transaction isolation levels.mp3


MoviePy - Done.
Audio extracted successfully for /content/videos/Recoverable schdeules and transaction isolation levels.mp4.
MoviePy - Writing audio in /content/Audio/Part1 - Definition of Information Security.mp3


MoviePy - Done.
Audio extracted successfully for /content/videos/Part1 - Definition of Information Security.mp4.


### Transcript it ( converting voice to text )

In [ ]:
output_Audio='/content/transcript/'
if not os.path.exists(output_Audio):
        os.makedirs(output_Audio)

In [ ]:
model = whisper.load_model("base")
for i in video_path:

  result = model.transcribe(i.replace("videos","Audio").split(".")[0]+".mp3")
  df_subtitles = pd.DataFrame(result["segments"]).drop(columns=["tokens","temperature","avg_logprob","compression_ratio","no_speech_prob","id","seek"])
  df_subtitles.to_csv("/content/transcript/"+i.split("/")[-1].split(".")[0]+".csv")




100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 48.0MiB/s]
  checkpoint = torch.load(fp, map_location=device)

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



## preprocessing on the data Removing stop word and celan it

In [ ]:
def removeStopWords(text):
  nltk.download('stopwords')
  stop_words = set(stopwords.words("english"))
  text = text.split()
  newText = ""
  for word in text:
    if word not in stop_words:
      newText += word+" "
  return newText.strip()

def applyFuncs(Data, textLabel):
  Data[textLabel] = Data[textLabel].apply(lambda x : x.lower())
  Data[textLabel] = Data[textLabel].apply(lambda x : re.sub("[^a-z0-9\s]","",x))
  Data[textLabel] = Data[textLabel].apply(lambda x : re.sub("\s+"," ",x))
  Data[textLabel] = Data[textLabel].apply(lambda x : removeStopWords(x))
  return Data

### using a transformer to extract the important data and summarize it

In [ ]:

tokenizer = BertTokenizer.from_pretrained("kairaamilanii/BERT2BERT-ExtractiveSummarizer")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("kairaamilanii/BERT2BERT-ExtractiveSummarizer")


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/999M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
def summarize_text(text, chunk_size=512):

    inputs = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    summary = ""

    for chunk in inputs:
        input_ids = tokenizer.encode(chunk, return_tensors='pt')

        summary_ids = model.generate(
            input_ids,
            min_length=128,
            max_length=256,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample=True,
            temperature=0.8,
            top_k=50,
            top_p=0.95
        )

        summary += tokenizer.decode(summary_ids[0], skip_special_tokens=True) + " "

    return summary


In [ ]:
subtitles=glob("/content/transcript/*.csv")

In [ ]:
for i in subtitles:
  DATA=pd.read_csv(i)
  DATA=applyFuncs(DATA,"text")
  segment_text = "dot ".join(DATA['text'].tolist())
  summary = summarize_text(segment_text)
  all_sum= summary.split("dot ")
  filtered_df = DATA[DATA['text'].isin(all_sum)]
  print(filtered_df)
  all_text = DATA["text"].tolist() + all_sum
  vectorizer = TfidfVectorizer().fit_transform(all_text)
  vectors = vectorizer.toarray()
  similarity_matrix = cosine_similarity(vectors[:len(DATA)], vectors[len(DATA):])
  df=DATA.copy()
  threshold = 0.4
  df["best_match"] = ""
  df["similarity_score"] = 0.0

  for j, subtitle_vector in enumerate(similarity_matrix):
    best_match_idx = subtitle_vector.argmax()
    best_score = subtitle_vector[best_match_idx]
    if best_score > threshold:
        df.at[j, "best_match"] = all_sum[best_match_idx]
        df.at[j, "similarity_score"] = best_score

  matched_df = df[df["similarity_score"] > threshold]
  matched_df.to_csv(i.split("/")[-1].split(".")[0]+"_summary.csv")






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_

    Unnamed: 0   start     end  \
0            0    0.00   23.20   
1            1   23.20   31.32   
24          24  207.96  216.20   
48          48  401.32  409.40   

                                                 text  
0   hello welcome first weeks lecture course intro...  
1   systems week building platform course upon fol...  
24  operating system detail let us start simple pr...  
48  perhaps way coordinates set depth color set st...  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /ro

     Unnamed: 0   start     end  \
34           34   451.6   458.6   
51           51   597.6   603.6   
59           59   675.6   677.6   
64           64   715.6   718.6   
92           92  1071.6  1083.6   
119         119  1278.6  1287.6   
125         125  1340.6  1351.6   

                                                  text  
34                 know undo lot efforts done database  
51                                  surely commit come  
59                                 take corrective act  
64                                           intuition  
92   actually dangerous sql take extra precaution a...  
119  things cannot antitiped things take extra prec...  
125  using techniques able solve issue phantoms con...  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /ro

     Unnamed: 0   start     end  \
0             0    0.00   22.96   
10           10   82.24   92.80   
11           11   92.80   99.28   
17           17  138.96  145.20   
51           51  372.16  377.12   
76           76  507.92  513.92   
120         120  763.52  768.08   
137         137  856.32  862.64   

                                                  text  
0    hello riba welcome nptel course introduction m...  
10   mitchell 97 machine agent deliberately leave b...  
11   could also apply non machines like biological ...  
17   going define learning second thing need kind p...  
51   going different machine learning paradigms tal...  
76   task right popular unsupervised learning parad...  
120  answering second question choose model right g...  
137  intuition behind necessarily questions buildin...  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /ro

     Unnamed: 0   start     end  \
0             0    0.00   10.00   
1             1   10.00   12.00   
2             2   12.00   15.00   
3             3   15.00   18.00   
4             4   18.00   20.00   
..          ...     ...     ...   
265         265  748.00  749.00   
267         267  750.68  752.36   
282         282  829.60  834.08   
293         293  889.44  894.60   
294         294  894.60  898.92   

                                                  text  
0                                                       
1                                                       
2                                                       
3                                                       
4                                                       
..                                                 ...  
265                                                     
267                                                  q  
282                         need interconnect networks  
293      